In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Tải các thư viện cần thiết

In [ ]:
!pip install -q mediapipe pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have pro

#Import thư viện

In [ ]:
import mediapipe as mp
import cv2
import os
import numpy as np
import pandas as pd

#Khởi tạo đường dẫn

In [ ]:
SOURCE_DATA_PATH = '/content/drive/MyDrive/Sign Language Recognition System/data/raw_data/Sign Language for Alphabets'
OUTPUT_CSV_PATH = '/content/drive/MyDrive/Sign Language Recognition System/processed/landmark_dataset.csv'

print("Đường dẫn nguồn:", SOURCE_DATA_PATH)
print("Đường dẫn đích:", OUTPUT_CSV_PATH)

Đường dẫn nguồn: /content/drive/MyDrive/Sign Language Recognition System/data/raw_data/Sign Language for Alphabets
Đường dẫn đích: /content/drive/MyDrive/Sign Language Recognition System/processed/landmark_dataset.csv


#Tiền xử lí dữ liệu

In [ ]:
all_items = os.listdir(SOURCE_DATA_PATH)
# Lọc ra chỉ những thư mục, bao gồm cả 'unknown'
class_folders = sorted([item for item in all_items if os.path.isdir(os.path.join(SOURCE_DATA_PATH, item)) and not item.startswith('.')])

print("Các thư mục lớp đã được lọc sạch:", class_folders)
print("Tổng số lớp tìm thấy:", len(class_folders))


Các thư mục lớp đã được lọc sạch: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'unknown', 'v', 'w', 'x', 'y', 'z']
Tổng số lớp tìm thấy: 27


## Khởi tạo mediapipe

In [ ]:
mp_hands = mp.solutions.hands

# Giảm min_detection_confidence để thử bắt được nhiều tay hơn
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.3)

data = []
labels = []
class_names = class_folders # Sử dụng danh sách đã lọc
label_map = {name: i for i, name in enumerate(class_names)}

In [ ]:
print("Các lớp:", class_names)
print("Nhãn:", label_map)
print("Tổng số lớp:", len(class_names))
print("Tổng số nhãn:", len(label_map))

Các lớp: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'unknown', 'v', 'w', 'x', 'y', 'z']
Nhãn: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'unknown': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
Tổng số lớp: 27
Tổng số nhãn: 27


##Trích Xuất Đặc Trưng Từ Ảnh Bàn Tay Bằng MediaPipe

In [ ]:
# Số lượng đặc trưng = 21 điểm mốc * 3 tọa độ (x, y, z)
NUM_FEATURES = 21 * 3

for class_name in class_names:
    class_dir = os.path.join(SOURCE_DATA_PATH, class_name)
    if not os.path.isdir(class_dir): continue
    print(f"Đang xử lý lớp: {class_name}")

    image_files = os.listdir(class_dir)
    for img_name in image_files:
        img_path = os.path.join(class_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            print(f"  >> Cảnh báo: không thể đọc ảnh {img_path}")
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            # Nếu tìm thấy tay, xử lý như bình thường
            hand_landmarks = results.multi_hand_landmarks[0]

            landmarks_row = []
            for landmark in hand_landmarks.landmark:
                landmarks_row.extend([landmark.x, landmark.y, landmark.z])

            # Chuẩn hóa
            wrist_x, wrist_y, wrist_z = landmarks_row[0], landmarks_row[1], landmarks_row[2]
            relative_landmarks = []
            for i in range(0, len(landmarks_row), 3):
                relative_landmarks.extend([
                    landmarks_row[i] - wrist_x,
                    landmarks_row[i+1] - wrist_y,
                    landmarks_row[i+2] - wrist_z
                ])

            data.append(relative_landmarks)
            labels.append(label_map[class_name])
        else:
            # Nếu KHÔNG tìm thấy tay
            # Chỉ thêm vector 0 nếu ảnh thuộc thư mục "unknown"
            if class_name == 'unknown':
                zero_landmarks = [0.0] * NUM_FEATURES
                data.append(zero_landmarks)
                labels.append(label_map[class_name])
            # Nếu không tìm thấy tay trong các thư mục chữ cái, chúng ta bỏ qua
            # vì đó là lỗi nhận diện, không phải lớp 'unknown'

# Đóng tài nguyên của MediaPipe
hands.close()

Đang xử lý lớp: a
Đang xử lý lớp: b
Đang xử lý lớp: c
Đang xử lý lớp: d
Đang xử lý lớp: e
Đang xử lý lớp: f
Đang xử lý lớp: g
Đang xử lý lớp: h
Đang xử lý lớp: i
Đang xử lý lớp: j
Đang xử lý lớp: k
Đang xử lý lớp: l
Đang xử lý lớp: m
Đang xử lý lớp: n
Đang xử lý lớp: o
Đang xử lý lớp: p
Đang xử lý lớp: q
Đang xử lý lớp: r
Đang xử lý lớp: s
Đang xử lý lớp: t
Đang xử lý lớp: u
Đang xử lý lớp: unknown
Đang xử lý lớp: v
Đang xử lý lớp: w
Đang xử lý lớp: x
Đang xử lý lớp: y
Đang xử lý lớp: z


#Lưu dữ liệu đã qua xử lí

In [ ]:
df_data = pd.DataFrame(data)
df_labels = pd.Series(labels, name='label')
final_df = pd.concat([df_labels, df_data], axis=1)

# Tạo thư mục nếu chưa tồn tại
output_dir = os.path.dirname(OUTPUT_CSV_PATH)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

final_df.to_csv(OUTPUT_CSV_PATH, index=False)

print(f"\nHOÀN TẤT! Đã tạo bộ dữ liệu điểm mốc tại: {OUTPUT_CSV_PATH}")
print(f"Tổng số mẫu: {len(final_df)}")
print("\nPhân bố các lớp:")
print(final_df['label'].value_counts().sort_index())


HOÀN TẤT! Đã tạo bộ dữ liệu điểm mốc tại: /content/drive/MyDrive/Sign Language Recognition System/processed/landmark_dataset.csv
Tổng số mẫu: 10626

Phân bố các lớp:
label
0      512
1      587
2       49
3      656
4       33
5      838
6      375
7      406
8      147
9     1088
10     878
11     292
12      14
13      25
14     317
15     121
17     926
18     295
19      66
20     140
21    1500
22     551
23     599
24      30
25     155
26      26
Name: count, dtype: int64
